In [44]:
import random
import json
import os

class IPLTable:
    def __init__(self):
        self.teams = {"CSK": [10, 9, 0.5], "RCB": [6, 10, 0.2], "KXIP": [6, 9, -0.3],
                      "DC": [10, 11, 0.8], "SRH": [10, 9, 0.6], "MI": [6, 9, -0.1],
                      "GT": [8, 10, 0.3], "LSG": [10, 9, 0.4], "KKR": [12, 9, 0.7],
                      "RR": [16, 9, 0.9]}  # Dictionary to store team data: {team_name: [points, matches_played, NRR]}
        self.match_results = []

    def predict_matches(self, upcoming_matches):
        for match in upcoming_matches:
            team1, team2 = match
            winning_team = random.choice([team1, team2])
            if winning_team == team1:
                losing_team = team2
            else:
                losing_team = team1
            
            self.update_points(winning_team, losing_team)
            self.match_results.append({"match": match, "winner": winning_team})

    def update_points(self, winning_team, losing_team):
        self.teams[winning_team][0] += 2  # Increment points by 2 for the winning team
        self.teams[winning_team][1] += 1  # Increment matches played for the winning team
        self.teams[losing_team][1] += 1  # Increment matches played for the losing team

    def save_table(self, filename, directory):
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        filepath = os.path.join(directory, filename)
        with open(filepath, "w") as f:
            json.dump(self.teams, f)

    def save_match_results(self, filename, directory):
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        filepath = os.path.join(directory, filename)
        with open(filepath, "w") as f:
            json.dump(self.match_results, f)

def main():
    upcoming_matches = [("CSK", "KXIP"), ("SRH", "RR"), ("KKR", "MI"),
                       ("RCB", "GT"), ("KXIP", "CSK"), ("LSG", "KKR"),
                       ("MI", "SRH"), ("DC", "RR"), ("SRH", "LSG"),
                       ("KXIP", "RCB"), ("GT", "CSK"), ("KKR", "MI"),
                       ("CSK", "RR"), ("RCB", "DC"), ("GT", "KKR"),
                       ("DC", "LSG"), ("KXIP", "RR"), ("SRH", "GT"),
                       ("MI", "LSG"), ("RCB", "CSK"), ("SRH", "KXIP"),
                       ("RR", "KKR")]

    num_iterations = 10000  # Number of times to run the prediction
    table_directory = r"C:\Users\DAYDREAMER\Project\IPL Table\Tables"
    match_results_directory = r"C:\Users\DAYDREAMER\Project\IPL Table\Match Results"
    for i in range(num_iterations):
        ipl_table = IPLTable()
        ipl_table.predict_matches(upcoming_matches)
        filename = f"table_{i+1}.json"
        ipl_table.save_table(filename, table_directory)  # Save the table after each iteration
        match_results_filename = f"match_results_{i+1}.json"
        ipl_table.save_match_results(match_results_filename, match_results_directory)  # Save match results

if __name__ == "__main__":
    main()


In [46]:
from collections import defaultdict

def load_tables(directory):
    tables = []
    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            filepath = os.path.join(directory, filename)
            with open(filepath, "r") as f:
                table = json.load(f)
                tables.append(table)
    return tables

def calculate_probabilities(tables):
    num_iterations = len(tables)
    team_position_counts = defaultdict(lambda: defaultdict(int))

    for table in tables:
        sorted_teams = sorted(table.items(), key=lambda x: (x[1][0], x[1][1], x[1][2]), reverse=True)
        for position, (team, _) in enumerate(sorted_teams, start=1):
            team_position_counts[team][position] += 1

    probabilities = defaultdict(lambda: defaultdict(float))
    for team, position_counts in team_position_counts.items():
        for position, count in position_counts.items():
            probabilities[team][position] = count / num_iterations

    return probabilities

def main():
    directory = r"C:\Users\DAYDREAMER\Project\IPL Table\Tables"
    num_iterations = 10000
    tables = load_tables(directory)
    probabilities = calculate_probabilities(tables)

    print("Probability of each team in each position:")
    for team, positions in probabilities.items():
        print(f"{team}:")
        for position, probability in positions.items():
            print(f"Position {position}: {probability:.2f}")
        print()

if __name__ == "__main__":
    main()

Probability of each team in each position:
RR:
Position 1: 0.91
Position 2: 0.08
Position 3: 0.01
Position 4: 0.00

SRH:
Position 2: 0.14
Position 3: 0.31
Position 5: 0.15
Position 4: 0.20
Position 7: 0.06
Position 6: 0.09
Position 9: 0.01
Position 8: 0.02
Position 1: 0.01
Position 10: 0.00

KKR:
Position 3: 0.17
Position 2: 0.57
Position 4: 0.09
Position 5: 0.05
Position 6: 0.02
Position 1: 0.07
Position 8: 0.00
Position 7: 0.01
Position 9: 0.00

CSK:
Position 4: 0.26
Position 5: 0.19
Position 1: 0.01
Position 6: 0.10
Position 7: 0.06
Position 3: 0.22
Position 2: 0.11
Position 8: 0.03
Position 9: 0.01
Position 10: 0.00

KXIP:
Position 5: 0.06
Position 10: 0.19
Position 4: 0.03
Position 9: 0.25
Position 7: 0.15
Position 8: 0.22
Position 3: 0.02
Position 6: 0.09
Position 2: 0.00

DC:
Position 6: 0.21
Position 5: 0.20
Position 8: 0.07
Position 3: 0.13
Position 7: 0.12
Position 4: 0.20
Position 2: 0.05
Position 9: 0.03
Position 10: 0.00

LSG:
Position 7: 0.15
Position 6: 0.19
Position 4: 